In [ ]:
%matplotlib notebook

import pylab
pylab.rcParams['figure.figsize'] = (8.0, 8.0)
pylab.rcParams['image.cmap'] = 'rainbow'

import numpy
import scipy
import scipy.special

from msclean import *
from synthesis import *
from simulate import *

from matplotlib import pylab
from matplotlib import pyplot as plt

def aaf_ns(a, m, c):
    """

    """
    r=numpy.hypot(*ucs(a))
    return scipy.special.pro_ang1(m,m,c,r)

def stackshow(stack):
    plt.clf()
    nscales=stack.shape[2]
    for iscale in range(nscales):
        ny=int(numpy.sqrt(nscales))
        nx=ny
        plt.subplot(nx,ny,iscale+1)
        plt.imshow(stack[:,:,iscale],origin='lower',cmap='Greys')

In [ ]:
vlas=numpy.genfromtxt("test/VLA_A_hor_xyz.txt", delimiter=",")
vobs=genuv(vlas, numpy.arange(0,numpy.pi,0.1) ,  numpy.pi/4)
yy=genvis(vobs/5, 0.01, 0.01)

In [ ]:
ps, vs = sortw(vobs/5, yy)
wstep=250000
dirty,psf=doimg(2*0.025, 2*15000, ps, vs, lambda *x: wslicimg(*x, wstep=wstep, Qpx=1))

In [ ]:
scales = [1]
scaleshape=[dirty.shape[0], dirty.shape[1], len(scales)]
scalescaleshape=[dirty.shape[0], dirty.shape[1], len(scales), len(scales)]
scalestack=createscalestack(scaleshape,scales)
psfscalestack=convolvescalestack(scalestack, psf)
resscalestack=convolvescalestack(scalestack, dirty)

In [ ]:
psfscalescalestack=numpy.zeros(scalescaleshape)
for iscale in numpy.arange(len(scales)):
    psfscalescalestack[:,:,:,iscale]=convolvescalestack(scalestack,psfscalestack[:,:,iscale])
    psfscalescalestack[:,:,iscale,:]=psfscalescalestack[:,:,:,iscale]

In [ ]:
couplingMatrix=numpy.zeros([len(scales),len(scales)])
for iscale in numpy.arange(len(scales)):
    for iscale1 in numpy.arange(len(scales)):
        couplingMatrix[iscale,iscale1]=numpy.max(psfscalescalestack[:,:,iscale,iscale1])

print couplingMatrix

In [ ]:
ps, vs, cc, rres=majorcycle(2*0.025, 2*15000, vobs/5, yy, 0.1, 5, 100, 250000, scales)
    

In [ ]:
plt.clf()
plt.subplot(1, 2, 1)
plt.imshow(numpy.sqrt(numpy.fabs(cc)), cmap='Greys', origin='lower')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(rres, cmap='Greys', origin='lower')
plt.colorbar()

plt.show()

In [ ]:
vs.shape